<a href="https://colab.research.google.com/github/AlkaidCheng/example/blob/master/RandomForestHParamTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install  --upgrade bayesian-optimization
! pip install ray[tune]

     |████████████████████████████████| 19.1MB 160kB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 2.9MB 44.1MB/s 
     |████████████████████████████████| 1.2MB 33.5MB/s 
     |████████████████████████████████| 204kB 42.4MB/s 
     |████████████████████████████████| 256kB 32.0MB/s 
     |████████████████████████████████| 153kB 43.1MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=899367d39c54c2017e29937a504771ecb9c8cef9fd598310adff8588d7f006e1
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [2]:
!wget https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz

--2020-03-08 06:45:15--  https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz
Resolving gitlab.cern.ch (gitlab.cern.ch)... 188.185.68.13, 188.184.28.144, 188.184.30.144, ...
Connecting to gitlab.cern.ch (gitlab.cern.ch)|188.185.68.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9140 (8.9K) [application/zip]
Saving to: ‘QIS_EXAM_200Events.npz’

QIS_EXAM_200Events. 100%[===================>]   8.93K  --.-KB/s    in 0s      

2020-03-08 06:45:15 (98.7 MB/s) - ‘QIS_EXAM_200Events.npz’ saved [9140/9140]



In [0]:
import numpy as np

In [0]:
def shuffle_zippedarrays(arrays):
  shape = arrays[0].shape
  assert all(shape == arrays[0].shape for arr in arrays)
  index = np.random.permutation(shape[0])
  return [arr[index] for arr in arrays]

def shuffle_data(input, label):
  output = shuffle_zippedarrays([input,label])
  return output[0], output[1]

def load_data(arrays, shuffle = True):
  data = {}
  for key in arrays:
    input, label = np.array([]), np.array([])
    for klabel in arrays[key].item():
      _input = arrays[key].item()[klabel]
      _label = np.full((_input.shape[0],),int(klabel))
      input = np.concatenate((input,_input),axis=0) if input.size else _input
      label = np.concatenate((label,_label),axis=0) if label.size else _label
    if shuffle:
      input, label = shuffle_data(input, label)
    data[key] = {'input': input, 'label': label}
  return data
def load_train_test_input_labels(arrays, shuffle = True):
  data = load_data(arrays, shuffle)
  return data['training_input']['input'], data['test_input']['input'], data['training_input']['label'], data['test_input']['label']

In [0]:
data = np.load('QIS_EXAM_200Events.npz',allow_pickle=True)
train_input, test_input, train_label, test_label = load_train_test_input_labels(data)
X = np.concatenate((train_input,test_input),axis = 0)
Y = np.concatenate((train_label,test_label),axis = 0)

In [9]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import cross_val_score
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import ray
from hyperopt import hp
from ray import tune


def train_breast_cancer(params):
    clf = RandomForestClassifier(**params)
    accuracy_score = cross_val_score(clf, X, Y).mean()
    tune.track.log(
        mean_accuracy=accuracy_score,
        done=True)
    
ray.shutdown()
ray.init(webui_host='127.0.0.1')

num_threads = 2

space = {
    # Maximum number of levels in tree
    'max_depth': hp.choice('max_depth', range(1,100)),
    # Number of features to consider at every split
    'max_features': hp.choice('max_features', ['auto','sqrt']),
    # Number of trees in random forest
    'n_estimators': hp.choice('n_estimators', range(100,2000)),
    # Minimum number of samples required to split a node
    'min_samples_split' :hp.choice('min_samples_split', range(2,10)),
    # Minimum number of samples required at each leaf node
    'min_samples_leaf' :hp.choice('min_samples_leaf', range(1,10)),
    # Method of selecting samples for training each tree    
    'criterion': hp.choice('criterion', ["gini", "entropy"])
    }

algo = HyperOptSearch(
        space,
        max_concurrent=4,
        metric="mean_accuracy",
        mode="max")
from ray.tune.schedulers import ASHAScheduler
! rm -r /lightgbm_result
analysis = tune.run(
    train_breast_cancer,
    num_samples=100,
    verbose =  0,
    resources_per_trial={'gpu': 1},
    search_alg = algo,
    local_dir = '/randomforest_result',
    scheduler=AsyncHyperBandScheduler(metric="mean_accuracy", mode="max"))
print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

2020-03-08 06:51:59,669	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-08 06:51:59,674	INFO resource_spec.py:212 -- Starting Ray with 6.59 GiB memory available for workers and up to 3.3 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-08 06:51:59,978	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265


rm: cannot remove '/lightgbm_result': No such file or directory


2020-03-08 06:52:06,121	INFO function_runner.py:250 -- tune.track signature detected.
2020-03-08 06:59:10,529	WARNING worker.py:1058 -- The actor or task with ID ffffffffffffffff8845600d0100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000}, {GPU: 1.000000} for execution and {CPU: 1.000000}, {GPU: 1.000000} for placement, but this node only has remaining {GPU: 1.000000}, {node:172.28.0.2: 1.000000}, {CPU: 2.000000}, {memory: 6.591797 GiB}, {object_store_memory: 2.246094 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.
2020-03-08 07:12:35,512	WARNING import_thread.py:136 -- The actor 'WrappedTrackFunc' has been exported 100 times. It's possible that this warning is accidental, but this m

Best config:  {'criterion': 'gini', 'max_depth': 1, 'max_features': 'auto', 'min_samples_leaf': 5, 'min_samples_split': 6, 'n_estimators': 280}
